In [ ]:
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

from langchain.chat_models import init_chat_model
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate
from pydantic import BaseModel
import mlflow
import os

In [2]:
#mlflow.langchain.autolog()  # For LangChain

In [ ]:
from getpass import getpass

os.environ['GOOGLE_API_KEY'] = getpass("Enter your Google API key: ")
print("Key loaded!")

In [4]:
model = init_chat_model(model="gemini-2.5-flash", model_provider="google_genai", max_tokens=500)

In [5]:
class ContactInfo(BaseModel):
    name: str
    email: str
    phone: str

In [6]:
# Create a prompt template for contact extraction
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert at extracting contact information from text. Extract the name, email, and phone number and return them in JSON format with keys: name, email, phone."),
    ("human", "{text}")
])

# Create a Chain that MLflow can log
contact_chain = LLMChain(llm=model, prompt=prompt)

/tmp/ipykernel_78098/1923082509.py:8: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  contact_chain = LLMChain(llm=model, prompt=prompt)


In [7]:
mlflow.set_experiment("Cooking Assistant")

<Experiment: artifact_location='file:///home/train/vbo-mlops/05_mlflow/notebooks/mlruns/578621774246543415', creation_time=1764471076350, experiment_id='578621774246543415', last_update_time=1764471076350, lifecycle_stage='active', name='Cooking Assistant', tags={}>

In [8]:
test_text = "John Doe, john@example.com, (555) 123-4567"

In [ ]:
with mlflow.start_run():
    # Log the LLMChain which MLflow supports
    model_info = mlflow.langchain.log_model(
        contact_chain, 
        name="contact_extraction_chain",
        input_example={"text": test_text}
    )
    
    # Test the chain using invoke (not deprecated)
    result = contact_chain.invoke({"text": test_text})
    print(f"Extraction result: {result}")
    
    # Log the result
    mlflow.log_text(str(result), "extraction_output.txt")

In [10]:
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

dish1 = loaded_model.predict({"recipe": "boeuf bourginon", "customer_count": "4"})

print(dish1[0])

ConstructorError: could not determine a constructor for the tag 'tag:yaml.org,2002:python/tuple'
  in "/home/train/vbo-mlops/05_mlflow/notebooks/mlruns/578621774246543415/models/m-5515123795f44d7faeb4c18d16f84217/artifacts/model.yaml", line 8, column 21

name='John Doe' email='john@example.com' phone='(555) 123-4567'
